In [1]:
#import libraries
import db_ops
import pandas as pd

Connecting to the PostgreSQL database...
Connection successful


## Load file with images, parse csv and fix some of the problems

In [2]:
df = pd.read_excel('./data/final_output_with_images.xlsx')
df = df.groupby('image_url', as_index=False).first() # Remove all duplicate images
df.sort_values(by = 'id', inplace= True)             # Sort by ID
df = df.rename(columns={'id':'olm_id'})              # Rename ID column to olm_id to not confuse in the DB
print(f"Total {df['image_url'].count()} images. {df['image_url'].nunique()} of them are unique")

Total 27388 images. 27388 of them are unique


In [3]:
#Subset a df that only has image metadata, we will deal with catregories later
pictures_df = df[['olm_id', 'verification', 'phone', 'date_taken', 'date_uploaded', 'lat', 'lon', 'picked up', 'address', 'total_litter', 'image_url']]

In [4]:
# Replace imgae url with the correct s3 uri
pictures_df['image_url'] = pictures_df['image_url'].apply(lambda x: 's3://olm-pics-s3/' + x.split('.com/')[-1])
pictures_df = pictures_df.rename(columns={'image_url': 's3_uri', 'picked up':'picked_up'})

# Convert picked up to bool
pictures_df = pictures_df.rename(columns={'picked up':'picked_up'})
pictures_df['picked_up'] = pictures_df['picked_up'].apply(lambda x: 1 if x == 'Yes' else 0).astype(bool)

# Convert total litter into an integer
pictures_df['total_litter'] = pictures_df['total_litter'].fillna(0)
pictures_df['total_litter'] = pictures_df['total_litter'].astype(int)

# Sanitize address string
pictures_df['address'] = pictures_df['address'].apply(lambda x: x.replace("'", ""))
pictures_df.head(3)

/tmp/ipykernel_272753/1241174448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pictures_df['image_url'] = pictures_df['image_url'].apply(lambda x: 's3://olm-pics-s3/' + x.split('.com/')[-1])


,olm_id,verification,phone,date_taken,date_uploaded,lat,lon,picked_up,address,total_litter,s3_uri
32,4222,2,SAMSUNG-SM-J727A,12/12/17 11:27,1/25/18 17:59,29.456944,-98.422500,False,"Warrior and Family Support Center, George Beac...",4,s3://olm-pics-s3/112/Pftq4550WzgZA5euhgIEsLHGs...
21,4223,2,SAMSUNG-SM-J727A,12/11/17 15:04,1/25/18 17:59,29.565833,-98.593889,False,"Silicon Drive, San Antonio, Bexar County, Texa...",3,s3://olm-pics-s3/112/HKXafwXvrpBQjoWC9aPtSY8lT...
6,4227,2,Unknown,1/25/18 18:24,1/25/18 18:24,29.455317,-98.424719,False,"Binz-Engleman Road, San Antonio, Bexar County,...",5,s3://olm-pics-s3/112/69M58FGWNVp1ZTObF0YmkMfGN...


In [11]:
# Create stage table for images 
db_ops.connection.rollback()
query = """

drop table if exists image;

CREATE TABLE "image"(image_id SERIAL PRIMARY KEY
                    ,olm_id INTEGER
                    ,verification SMALLINT
                    ,phone VARCHAR(30)
                    ,date_taken TIMESTAMP
                    ,date_uploaded TIMESTAMP
                    ,lat NUMERIC(15,12)
                    ,lon NUMERIC(15,12)
                    ,picked_up BOOL
                    ,address VARCHAR(255)
                    ,total_litter SMALLINT
                    ,s3_uri VARCHAR(255)
);

"""
db_ops.cursor.execute(query)
db_ops.connection.commit()

In [23]:
# Insert the df to this table
# Create a list of tupples from the dataframe values
tuples = [tuple(x) for x in pictures_df.to_numpy()]
# Comma-separated dataframe columns
cols = ','.join(list(pictures_df.columns))
# SQL quert to execute
query = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s, %%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % ("image", cols)
cursor = db_ops.connection.cursor()
cursor.executemany(query, tuples)
db_ops.connection.commit()


## Brands and categories tables

In [5]:
#Get a mixed list of categories and brands
br_cat_cols = list(set(df.columns)-set(pictures_df.columns) - set(['link', 'image_url', 'picked up', 'custom_tag_1', 'custom_tag_2', 'custom_tag_3']))

In [7]:
#Manually define what is a category
cats = ['filters',
 'glass_jar',
 'smoking_plastic',
 'beerCan',
 'aluminium',
 'aluminium_foil',
 'ceramic',
 'chemical',
 'small',
 'sanitaryOther',
 'rubber',
 'butts',
 'degraded_straws',
 'coffeeLids',
 'balloons',
 'pullring',
 'degraded_plasticbottle',
 'INDUSTRIAL',
 'milk_bottle',
 'other',
 'random_litter',
 'crisp_large',
 'vape_oil',
 'energy_can',
 'plastic_bags',
 'polymer',
 'FOOD',
 'traffic_cone',
 'titanium',
 'forsale_posters',
 'medium',
 'ice_tea_bottles',
 'skins',
 'microplastics',
 'dump',
 'poo',
 'tobaccoPouch',
 'life_buoy',
 'alcoholOther',
 'spiritBottle',
 'paper',
 'fishing_gear_nets',
 'rope_large',
 'glass',
 'napkins',
 'shotgun_cartridges',
 'sweetWrappers',
 'brokenGlass',
 'BRANDS',
 'plastic_cup_tops',
 'paperFoodPackaging',
 'tooth_pick',
 'COASTAL',
 'MATERIAL',
 'books',
 'coffeeOther',
 'ART',
 'paper_cups',
 'COFFEE',
 'milk_carton',
 'election_posters',
 'iron_or_steel',
 'wood',
 'paper.1',
 'cable_tie',
 'degraded_lighters',
 'sauce_packet',
 'large',
 'poo_in_bag',
 'broken_glass',
 'waterBottle',
 'rope_small',
 'rope_medium',
 'wineBottle',
 'vape_pen',
 'cigaretteBox',
 'latex',
 'hair_tie',
 'degraded_plasticbag',
 'metal',
 'metal.1',
 'thins',
 'softDrinkOther',
 'bags_litter',
 'stationary',
 'juice_bottles',
 'styro_medium',
 'copper',
 'polyethylene',
 'six_pack_rings',
 'crisp_small',
 'paperCardAlcoholPackaging',
 'concrete',
 'SMOKING',
 'styrofoam_plate',
 'industrial_other',
 'fiberglass',
 'OTHER',
 'straws',
 'DOGSHIT',
 'hand_sanitiser',
 'coastal_other',
 'batteries',
 'plastic_cups',
 'plasticCutlery',
 'industrial_plastic',
 'plasticFoodPackaging',
 'styro_large',
 'styro_small',
 'foodOther',
 'bottleLid',
 'plasticAlcoholPackaging',
 'polypropylene',
 'strawpacket',
 'lighters',
 'macroplastics',
 'composite',
 'deodorant',
 'beerBottle',
 'item',
 'pizza_box',
 'bottleLabel',
 'ice_tea_can',
 'automobile',
 'clothing',
 'juice_cartons',
 'carbon_fiber',
 'condoms',
 'styro_cup',
 'tape',
 'SOFTDRINKS',
 'chewing_gum',
 'coffeeCups',
 'SANITARY',
 'alcohol_plastic_cups',
 'buoys',
 'elec_small',
 'ear_swabs',
 'facemask',
 'tinCan',
 'plastic',
 'ALCOHOL',
 'oil',
 'pvc',
 'ear_plugs',
 'tooth_brush',
 'wetwipes',
 'gloves',
 'dogshit',
 'bulmers',
 'filterbox',
 'polystyrene',
 'DUMPING',
 'mediumplastics',
 'nickel',
 'elec_large',
 'smokingOther',
 'pint',
 'bronze',
 'bricks',
 'overflowing_bins',
 'nylon',
 'juice_packet',
 'bottleTops',
 'balloons.1',
 'glass_jar_lid',
 'sportsDrink',
 'magazine']

In [6]:
# Manually define what is a brand
brands = ['aadrink',
 'acadia',
 'adidas',
 'albertheijn',
 'aldi',
 'amazon',
 'amstel',
 'anheuser_busch',
 'apple',
 'applegreen',
 'asahi',
 'avoca',
 'bacardi',
 'ballygowan',
 'bewleys',
 'brambles',
 'budweiser',
 'bullit',
 'burgerking',
 'butlers',
 'cadburys',
 'cafe_nero',
 'calanda',
 'camel',
 'caprisun',
 'carlsberg',
 'centra',
 'circlek',
 'coke',
 'coles',
 'colgate',
 'corona',
 'costa',
 'doritos',
 'drpepper',
 'dunnes',
 'duracell',
 'durex',
 'esquires',
 'evian',
 'fanta',
 'fernandes',
 'fizzyDrinkBottle',
 'fosters',
 'frank_and_honest',
 'fritolay',
 'gatorade',
 'ghost_nets',
 'gillette',
 'goldenpower',
 'guinness',
 'haribo',
 'heineken',
 'hertog_jan',
 'insomnia',
 'kellogs',
 'kfc',
 'lavish',
 'lego',
 'lego.1',
 'lidl',
 'lindenvillage',
 'lipton',
 'lolly_and_cookes',
 'loreal',
 'lucozade',
 'marlboro',
 'mars',
 'mcdonalds',
 'menstral',
 'modelo',
 'molson_coors',
 'monster',
 'nappies',
 'nero',
 'nescafe',
 'nestle',
 'nike',
 'obriens',
 'ok_',
 'pepsi',
 'powerade',
 'redbull',
 'ribena',
 'sainsburys',
 'samsung',
 'schutters',
 'seven_eleven',
 'slammers',
 'spa',
 'spar',
 'starbucks',
 'stella',
 'subway',
 'supermacs',
 'supervalu',
 'tayto',
 'tesco',
 'tim_hortons',
 'tyre',
 'volvic',
 'waitrose',
 'walkers',
 'washing_up',
 'wendys',
 'wilde_and_greene',
 'winston',
 'woolworths',
 'wrigleys']

In [8]:
#Subset a df that only has categories and brands
br_cat_df = df[['olm_id'] + br_cat_cols]

# Convert the table from wide to long format and drop all records that have null value in value
# This way we will have an individual record for every occurance of a brand or category,
# with the number of the items of this category in the value column.
# Potentially many records for each image, with individual record for each brand that was observed in this image 
melt_df = pd.melt(br_cat_df, id_vars=['olm_id'], value_vars=br_cat_cols).dropna(subset=['value'])

In [9]:
# Code to check if a value is a number or not
'''
melt_df['is_number'] = melt_df['value'].apply(lambda x: str(x).replace('.', '', 1).isdigit())
melt_df[~melt_df['is_number']]
'''
# Convert value to ineger
melt_df['value'] = melt_df['value'].astype(int) 

In [13]:
# Create stage melted table for images 
#db_ops.connection.rollback()
query = """

drop table if exists melted_stage;

CREATE TABLE "melted_stage"(melt_id SERIAL PRIMARY KEY
                    ,olm_id INTEGER
                    ,variable VARCHAR(50)
                    ,value SMALLINT
);

"""
connection = db_ops.connect_db()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

Connecting to the PostgreSQL database...
Connection successful


In [14]:
# Insert the df to this table
# Create a list of tupples from the dataframe values
tuples = [tuple(x) for x in melt_df.to_numpy()]
# Comma-separated dataframe columns
cols = ','.join(list(melt_df.columns))
# SQL quert to execute
query = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % ("melted_stage", cols)
#cursor = connection.cursor()
cursor.executemany(query, tuples)
connection.commit()

In [16]:
# Create categories table for images 
#db_ops.connection.rollback()
query = """

drop table if exists category;

CREATE TABLE "category"(cat_id SERIAL PRIMARY KEY
                    ,cat_name VARCHAR(255)
);

"""
connection = db_ops.connect_db()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

cat_df = pd.DataFrame(cats, columns=['cat_name'])
db_ops.add_df(cat_df, "category")

Connecting to the PostgreSQL database...
Connection successful


In [45]:
# Create categories table for images 
#db_ops.connection.rollback()
query = """

drop table if exists category;

CREATE TABLE "category"(cat_id SERIAL PRIMARY KEY
                    ,cat_name VARCHAR(255)
);

"""
connection = db_ops.connect_db()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

cat_df = pd.DataFrame(cats, columns=['cat_name'])
db_ops.add_df(cat_df, "category")

,cat_name
0,filters
1,glass_jar
2,smoking_plastic


In [54]:
brand_df = pd.DataFrame(brands +['Snapple'], columns=['brand_name'])
brand_df ['score'] = 100

In [60]:
# Create categories table for images 
#db_ops.connection.rollback()
query = """

drop table if exists brand;

CREATE TABLE "brand"(brand_id SERIAL PRIMARY KEY
                    ,brand_name VARCHAR(50)
                    ,score NUMERIC
);

"""
connection = db_ops.connect_db()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

db_ops.add_df(brand_df, "brand")

Connecting to the PostgreSQL database...
Connection successful


In [57]:
recieved_name = 'Snapple'
query = f'''
SELECT * 
FROM brand as br
WHERE br.brand_name = '{recieved_name}';
'''

cursor.execute(query)
response_list = cursor.fetchall()
response_list

[(110, 'Snapple', Decimal('100'))]

In [59]:
import json
json.dumps({'brand_name':response_list[0][1], 'brand_score':float(response_list[0][2])})

'{"brand_name": "Snapple", "brand_score": 100.0}'